In [1]:
# Parameters
FUDGE_FACTOR = 1.1200  # Multiply forecasts by this

XGB_WEIGHT = 0.6200
BASELINE_WEIGHT = 0.0100
OLS_WEIGHT = 0.0620
NN_WEIGHT = 0.0800

XGB1_WEIGHT = 0.8000  # Weight of first in combination of two XGB models

BASELINE_PRED = 0.0115   # Baseline based on mean of training data, per Oleg

In [2]:
import numpy as np
import pandas as pd
import xgboost as xgb
import datetime as dt
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
import lightgbm as lgb
import gc
import random

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout, BatchNormalization
from keras.layers.advanced_activations import PReLU
from keras.layers.noise import GaussianDropout
from keras.optimizers import Adam
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

import warnings
warnings.filterwarnings("ignore")

In [3]:
train_prop = pd.read_csv("/Users/kimjw/Desktop/Kaggle/week6/properties_2016.csv")
train = pd.read_csv("/Users/kimjw/Desktop/Kaggle/week6/train_2016_v2.csv")

# Process Data

In [4]:
for col, dtype in zip(train_prop.columns, train_prop.dtypes):
    if dtype == "float64":
        train_prop[col] = train_prop[col].astype(np.float32)

In [5]:
train_df = train.merge(train_prop, on="parcelid",
                       how="left")
train_df.fillna(train_df.median(), inplace=True)
train_df.head()

,parcelid,logerror,transactiondate,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,11016594,0.0276,2016-01-01,1.0,7.0,616.0,2.0,3.0,4.0,4.0,...,1.0,1,122754.0,360170.0,2015.0,237416.0,6735.879883,NaN,14.0,6.037107e+13
1,14366692,-0.1684,2016-01-01,1.0,7.0,616.0,3.5,4.0,4.0,7.0,...,1.0,1,346458.0,585529.0,2015.0,239071.0,10153.019531,NaN,14.0,6.037615e+13
2,12098116,-0.0040,2016-01-01,1.0,7.0,616.0,3.0,2.0,4.0,4.0,...,1.0,1,61994.0,119906.0,2015.0,57912.0,11484.480469,NaN,14.0,6.037464e+13
3,12643413,0.0218,2016-01-02,1.0,7.0,616.0,2.0,2.0,4.0,4.0,...,1.0,1,171518.0,244880.0,2015.0,73362.0,3048.739990,NaN,14.0,6.037296e+13
4,14432541,-0.0050,2016-01-02,1.0,7.0,616.0,2.5,4.0,4.0,7.0,...,2.0,1,169574.0,434551.0,2015.0,264977.0,5488.959961,NaN,14.0,6.059042e+13


In [6]:
train_df.select_dtypes("object").columns

Index(['transactiondate', 'hashottuborspa', 'propertycountylandusecode',
       'propertyzoningdesc', 'fireplaceflag', 'taxdelinquencyflag'],
      dtype='object')

In [7]:
# 카테고리형 자료를 다 빼는 것 같은데 hashottuborspa도 빼야하지 않나?
x_train = train_df.drop(['parcelid', 'logerror', 'transactiondate', 'propertyzoningdesc', 
                         'propertycountylandusecode', 'fireplacecnt', 'fireplaceflag'], axis=1)
y_train = train_df['logerror'].values
print(x_train.shape, y_train.shape)

(90275, 53) (90275,)


In [8]:
train_columns = x_train.columns.tolist()

In [9]:
for col in x_train.dtypes[x_train.dtypes==object].index.values:
    x_train[col] = (x_train[col]==True)
x_train.head()

,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,...,yearbuilt,numberofstories,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,1.0,7.0,616.0,2.0,3.0,4.0,4.0,2.0,66.0,1244.0,...,1959.0,1.0,122754.0,360170.0,2015.0,237416.0,6735.879883,False,14.0,6.037107e+13
1,1.0,7.0,616.0,3.5,4.0,4.0,7.0,3.5,66.0,1244.0,...,2014.0,1.0,346458.0,585529.0,2015.0,239071.0,10153.019531,False,14.0,6.037615e+13
2,1.0,7.0,616.0,3.0,2.0,4.0,4.0,3.0,66.0,1244.0,...,1940.0,1.0,61994.0,119906.0,2015.0,57912.0,11484.480469,False,14.0,6.037464e+13
3,1.0,7.0,616.0,2.0,2.0,4.0,4.0,2.0,66.0,1244.0,...,1987.0,1.0,171518.0,244880.0,2015.0,73362.0,3048.739990,False,14.0,6.037296e+13
4,1.0,7.0,616.0,2.5,4.0,4.0,7.0,2.5,66.0,1244.0,...,1981.0,2.0,169574.0,434551.0,2015.0,264977.0,5488.959961,False,14.0,6.059042e+13


In [10]:
# garbage collection
del train_df; gc.collect()

42

In [11]:
x_train = x_train.values.astype(np.float32, copy=False)
x_train

array([[1.0000000e+00, 7.0000000e+00, 6.1600000e+02, ..., 0.0000000e+00,
        1.4000000e+01, 6.0371067e+13],
       [1.0000000e+00, 7.0000000e+00, 6.1600000e+02, ..., 0.0000000e+00,
        1.4000000e+01, 6.0376150e+13],
       [1.0000000e+00, 7.0000000e+00, 6.1600000e+02, ..., 0.0000000e+00,
        1.4000000e+01, 6.0374636e+13],
       ...,
       [1.0000000e+00, 7.0000000e+00, 6.1600000e+02, ..., 0.0000000e+00,
        1.4000000e+01, 6.0374326e+13],
       [1.0000000e+00, 7.0000000e+00, 6.1600000e+02, ..., 0.0000000e+00,
        1.4000000e+01, 6.0376012e+13],
       [1.0000000e+00, 7.0000000e+00, 6.1600000e+02, ..., 0.0000000e+00,
        1.4000000e+01, 6.0375442e+13]], dtype=float32)

In [12]:
d_train = lgb.Dataset(x_train, label=y_train)

# LightGBM

In [13]:
params = {}
params["max_bin"] = 10
params["learning_rate"] = 0.0021
params["boosting_type"] = "gbdt"
params["objective"] = "regression"
params["metric"] = "l1"
params["sub_feature"] = 0.345
params["bagging_fraction"] = 0.85
params["bagging_freq"] = 40
params["num_leaves"] = 512
params["min_data"] = 500
params["min_hessian"] = 0.05
params["verbose"] = 0
params["feature_fraction_seed"] = 2
params["bagging_seed"] = 3

np.random.seed(0)
random.seed(0)

# Fitting LGBM model
clf = lgb.train(params, d_train, 430)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007196 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] 

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [14]:
del d_train; gc.collect()
del x_train; gc.collect()

0

In [15]:
# Prepare for LightGBM prediction
sample = pd.read_csv("/Users/kimjw/Desktop/Kaggle/week6/sample_submission.csv")
sample["parcelid"] = sample["ParcelId"]
test_df = sample.merge(train_prop, on="parcelid", how="left")
test_df

,ParcelId,201610,201611,201612,201710,201711,201712,parcelid,airconditioningtypeid,architecturalstyletypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,10754147,0,0,0,0,0,0,10754147,NaN,NaN,...,NaN,NaN,NaN,9.0,2015.0,9.0,NaN,NaN,NaN,NaN
1,10759547,0,0,0,0,0,0,10759547,NaN,NaN,...,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN
2,10843547,0,0,0,0,0,0,10843547,NaN,NaN,...,NaN,NaN,650756.0,1413387.0,2015.0,762631.0,20800.369141,NaN,NaN,NaN
3,10859147,0,0,0,0,0,0,10859147,NaN,NaN,...,1.0,NaN,571346.0,1156834.0,2015.0,585488.0,14557.570312,NaN,NaN,NaN
4,10879947,0,0,0,0,0,0,10879947,NaN,NaN,...,NaN,NaN,193796.0,433491.0,2015.0,239695.0,5725.169922,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2985212,168176230,0,0,0,0,0,0,168176230,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2985213,14273630,0,0,0,0,0,0,14273630,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2985214,168040630,0,0,0,0,0,0,168040630,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2985215,168040830,0,0,0,0,0,0,168040830,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
del sample, train_prop; gc.collect()

20

In [17]:
x_test = test_df[train_columns]

NameError: name 'x_test' is not defined

In [18]:
for col in x_test.dtypes[x_test.dtypes == object].index.values:
    x_test[col] = (x_test[col] == True)

In [19]:
x_test = x_test.values.astype(np.float32, copy=False)
x_test

array([[nan, nan, nan, ...,  0., nan, nan],
       [nan, nan, nan, ...,  0., nan, nan],
       [nan, nan, nan, ...,  0., nan, nan],
       ...,
       [nan, nan, nan, ...,  0., nan, nan],
       [nan, nan, nan, ...,  0., nan, nan],
       [nan, nan, nan, ...,  0., nan, nan]], dtype=float32)

In [20]:
p_test = clf.predict(x_test)
p_test

array([0.03113213, 0.03337514, 0.010257  , ..., 0.02501801, 0.02501801,
       0.02501801])

In [21]:
del x_test; gc.collect()
print(pd.DataFrame(p_test))

                0
0        0.031132
1        0.033375
2        0.010257
3        0.008651
4        0.009660
...           ...
2985212  0.025018
2985213  0.025018
2985214  0.025018
2985215  0.025018
2985216  0.025018

[2985217 rows x 1 columns]


In [29]:
test_df.shape

(2985217, 65)

# XGBoost

In [30]:
properties = pd.read_csv("/Users/kimjw/Desktop/Kaggle/week6/properties_2016.csv")

In [31]:
for col in properties.columns:
    properties[col]=properties[col].fillna(-1)
    if properties[col].dtype == "object":
        lbl = LabelEncoder()
        lbl.fit(list(properties[col].values))
        properties[col] = lbl.transform(list(properties[col].values))

In [34]:
train_df = train.merge(properties, on="parcelid", how="left")
x_train = train_df.drop(["parcelid","logerror","transactiondate"], axis=1)
x_test = properties.drop(["parcelid"], axis=1)

In [39]:
print('Shape train: {}\nShape test: {}'.format(x_train.shape, x_test.shape))

Shape train: (90275, 57)
Shape test: (2985217, 57)


In [44]:
# 이상치 제거
train_df = train_df[train_df.logerror > -0.4]
train_df = train_df[train_df.logerror < 0.419]
train_df["logerror"].describe()

count    88528.000000
mean         0.007628
std          0.083459
min         -0.399000
25%         -0.025300
50%          0.005000
75%          0.037300
max          0.418700
Name: logerror, dtype: float64

In [45]:
x_train = train_df.drop(["parcelid","logerror","transactiondate"],axis=1)
y_train = train_df["logerror"].values.astype(np.float32)
y_mean = np.mean(y_train)

In [46]:
print('After removing outliers:')     
print('Shape train: {}\nShape test: {}'.format(x_train.shape, x_test.shape))

After removing outliers:
Shape train: (88528, 57)
Shape test: (2985217, 57)


In [47]:
xgb_params = {
    "eta": 0.037,
    "max_depth": 5,
    "subsample": 0.8,
    "objective": "reg:linear",
    "eval_metric": "mae",
    "lambda": 0.8,
    "alpha": 0.4,
    "base_score": y_mean,
    "silent": 1
}

dtrain = xgb.DMatrix(x_train, y_train)
dtest = xgb.DMatrix(x_test)

In [49]:
num_boost_rounds = 250
print("num_boost_rounds="+str(num_boost_rounds))

num_boost_rounds=250


In [50]:
# train model
print("\nTraining XGBoost ...")
model = xgb.train(dict(xgb_params), dtrain, num_boost_round=num_boost_rounds)


Training XGBoost ...
[12:55:49] WARNING: /Users/travis/build/dmlc/xgboost/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[12:55:49] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




In [51]:
print("\nPredicting with XGBoost ... ")
xgb_pred1 = model.predict(dtest)


Predicting with XGBoost ... 


In [52]:
print("\nFirst XGBoost predictions: ")
print(pd.DataFrame(xgb_pred1))


First XGBoost predictions: 
                0
0       -0.030616
1       -0.028188
2        0.026397
3        0.063728
4        0.004398
...           ...
2985212  0.097847
2985213  0.097847
2985214  0.097847
2985215  0.097847
2985216  0.097847

[2985217 rows x 1 columns]


In [53]:
# 새로운 파라미터로 재모델링
# xgboost params
xgb_params = {
    'eta': 0.033,
    'max_depth': 6,
    'subsample': 0.80,
    'objective': 'reg:linear',
    'eval_metric': 'mae',
    'base_score': y_mean,
    'silent': 1
}

num_boost_rounds = 150
print("num_boost_rounds="+str(num_boost_rounds))

print( "\nTraining XGBoost again ...")
model = xgb.train(dict(xgb_params), dtrain, num_boost_round=num_boost_rounds)

print( "\nPredicting with XGBoost again ...")
xgb_pred2 = model.predict(dtest)

print( "\nSecond XGBoost predictions:" )
print( pd.DataFrame(xgb_pred2) )

num_boost_rounds=150

Training XGBoost again ...
[12:58:50] WARNING: /Users/travis/build/dmlc/xgboost/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[12:58:50] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.



Predicting with XGBoost again ...

Second XGBoost predictions:
                0
0       -0.091150
1       -0.034722
2        0.015816
3        0.075518
4        0.029908
...           ...
2985212  0.062648
2985213  0.062648
2985214  0.062648
2985215  0.062648
2985216  0.062648

[2985217 rows x 1 columns]


In [54]:
# Combine XGBoost Result
# XGB1_WEIGHT=0.8000
xgb_pred = XGB1_WEIGHT*xgb_pred1 + (1-XGB1_WEIGHT)*xgb_pred2

print( "\nCombined XGBoost predictions:" )
print( pd.DataFrame(xgb_pred) )


Combined XGBoost predictions:
                0
0       -0.042723
1       -0.029495
2        0.024281
3        0.066086
4        0.009500
...           ...
2985212  0.090807
2985213  0.090807
2985214  0.090807
2985215  0.090807
2985216  0.090807

[2985217 rows x 1 columns]


In [55]:
del train_df
del x_train
del x_test
del properties
del dtest
del dtrain
del xgb_pred1
del xgb_pred2 
gc.collect()

4162

# Neural Network

In [104]:
prop = pd.read_csv("/Users/kimjw/Desktop/Kaggle/week6/properties_2016.csv")
train = pd.read_csv("/Users/kimjw/Desktop/Kaggle/week6/train_2016_v2.csv")
sample = pd.read_csv("/Users/kimjw/Desktop/Kaggle/week6/sample_submission.csv")

In [105]:
for c in prop.columns:
    prop[c]=prop[c].fillna(-1)
    if prop[c].dtype == "object":
        lbl=LabelEncoder()
        lbl.fit(list(prop[c].values))
        prop[c] = lbl.transform(list(prop[c].values))

In [106]:
df_train = train.merge(prop, on="parcelid", how="left")

In [107]:
df_train["transactiondate"]=pd.to_datetime(df_train["transactiondate"])
df_train["transactiondate_year"]=df_train["transactiondate"].dt.year
df_train["transactiondate_month"]=df_train["transactiondate"].dt.month
df_train["transactiondate_quarter"]=df_train["transactiondate"].dt.quarter
df_train["transactiondate"]=df_train["transactiondate"].dt.day

In [108]:
# filling NA/NaN values with -1.0 
df_train.fillna(-1.0)

,parcelid,logerror,transactiondate,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,...,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,transactiondate_year,transactiondate_month,transactiondate_quarter
0,11016594,0.0276,1,1.0,-1.0,-1.0,2.0,3.0,-1.0,4.0,...,360170.0,2015.0,237416.0,6735.88,0,-1.0,6.037107e+13,2016,1,1
1,14366692,-0.1684,1,-1.0,-1.0,-1.0,3.5,4.0,-1.0,-1.0,...,585529.0,2015.0,239071.0,10153.02,0,-1.0,-1.000000e+00,2016,1,1
2,12098116,-0.0040,1,1.0,-1.0,-1.0,3.0,2.0,-1.0,4.0,...,119906.0,2015.0,57912.0,11484.48,0,-1.0,6.037464e+13,2016,1,1
3,12643413,0.0218,2,1.0,-1.0,-1.0,2.0,2.0,-1.0,4.0,...,244880.0,2015.0,73362.0,3048.74,0,-1.0,6.037296e+13,2016,1,1
4,14432541,-0.0050,2,-1.0,-1.0,-1.0,2.5,4.0,-1.0,-1.0,...,434551.0,2015.0,264977.0,5488.96,0,-1.0,6.059042e+13,2016,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90270,10774160,-0.0356,30,1.0,-1.0,-1.0,1.0,1.0,-1.0,4.0,...,191000.0,2015.0,147200.0,2495.24,0,-1.0,6.037132e+13,2016,12,4
90271,12046695,0.0070,30,-1.0,-1.0,-1.0,3.0,3.0,-1.0,4.0,...,161111.0,2015.0,43218.0,1886.54,0,-1.0,6.037301e+13,2016,12,4
90272,12995401,-0.2679,30,-1.0,-1.0,-1.0,2.0,4.0,-1.0,7.0,...,38096.0,2015.0,16088.0,1925.70,1,14.0,6.037433e+13,2016,12,4
90273,11402105,0.0602,30,-1.0,-1.0,-1.0,2.0,2.0,-1.0,4.0,...,165869.0,2015.0,32878.0,2285.57,0,-1.0,6.037601e+13,2016,12,4


In [109]:
# create x_train, y_train from df_train
x_train = df_train.drop(["parcelid","logerror","transactiondate",
                         'propertyzoningdesc', 'propertycountylandusecode','fireplacecnt', 'fireplaceflag'], axis=1)
y_train = df_train["logerror"]
y_mean = np.mean(y_train)
print(x_train.shape, y_train.shape)

(90275, 56) (90275,)


In [110]:
train_columns = x_train.columns

In [111]:
for c in x_train.dtypes[x_train.dtypes == object].index.values:
    x_train[c] = (x_train[c]==True)

In [112]:
# create df_test
sample["parcelid"] = sample["ParcelId"]

In [113]:
# merge sample with property data
df_test = sample.merge(prop, on="parcelid", how="left")

In [114]:
df_test["transactiondate"] = pd.to_datetime('2016-11-15')  # placeholder value for preliminary version
df_test["transactiondate_year"] = df_test["transactiondate"].dt.year
df_test["transactiondate_month"] = df_test["transactiondate"].dt.month
df_test['transactiondate_quarter'] = df_test['transactiondate'].dt.quarter
df_test["transactiondate"] = df_test["transactiondate"].dt.day     

In [115]:
x_test = df_test[train_columns]

In [116]:
print('Shape of x_test:', x_test.shape)
print("Preparing x_test...")
for c in x_test.dtypes[x_test.dtypes == object].index.values:
    x_test[c] = (x_test[c] == True)

Shape of x_test: (2985217, 56)
Preparing x_test...


In [117]:
# preprocessing
imputer = SimpleImputer()
imputer.fit(x_train.iloc[:,:])
x_train = imputer.transform(x_train.iloc[:,:])
imputer.fit(x_test.iloc[:,:])
x_test = imputer.transform(x_test.iloc[:,:])

In [118]:
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

len_x=int(x_train.shape[1])
print("len_x is: ",len_x)

len_x is:  56


In [120]:
# Neural Network
print("Setting up neural network model...")
nn = Sequential()
nn.add(Dense(units=400, kernel_initializer="normal",
            input_dim=len_x))
nn.add(PReLU())
nn.add(Dropout(.4))
nn.add(Dense(units=400, kernel_initializer="normal"))
nn.add(PReLU())
nn.add(BatchNormalization())
nn.add(Dropout(.6))
nn.add(Dense(units=64, kernel_initializer="normal"))
nn.add(PReLU())
nn.add(BatchNormalization())
nn.add(Dropout(.5))
nn.add(Dense(units=26, kernel_initializer="normal"))
nn.add(PReLU())
nn.add(BatchNormalization())
nn.add(Dropout(.6))
nn.add(Dense(1, kernel_initializer="normal"))
nn.compile(loss="mae", optimizer=Adam(lr=4e-3, decay=1e-4))

Setting up neural network model...


In [123]:
print("Fitting neural network model ...")
nn.fit(np.array(x_train), np.array(y_train),
      batch_size=32, epochs=70, verbose=2)

Fitting neural network model ...
Epoch 1/70
2822/2822 - 22s - loss: 0.0712
Epoch 2/70
2822/2822 - 17s - loss: 0.0682
Epoch 3/70
2822/2822 - 21s - loss: 0.0682
Epoch 4/70
2822/2822 - 22s - loss: 0.0681
Epoch 5/70
2822/2822 - 22s - loss: 0.0680
Epoch 6/70
2822/2822 - 23s - loss: 0.0680
Epoch 7/70
2822/2822 - 23s - loss: 0.0679
Epoch 8/70
2822/2822 - 25s - loss: 0.0679
Epoch 9/70
2822/2822 - 25s - loss: 0.0678
Epoch 10/70
2822/2822 - 22s - loss: 0.0678
Epoch 11/70
2822/2822 - 23s - loss: 0.0678
Epoch 12/70
2822/2822 - 23s - loss: 0.0678
Epoch 13/70
2822/2822 - 23s - loss: 0.0677
Epoch 14/70
2822/2822 - 23s - loss: 0.0677
Epoch 15/70
2822/2822 - 23s - loss: 0.0676
Epoch 16/70
2822/2822 - 23s - loss: 0.0676
Epoch 17/70
2822/2822 - 23s - loss: 0.0676
Epoch 18/70
2822/2822 - 22s - loss: 0.0676
Epoch 19/70
2822/2822 - 23s - loss: 0.0675
Epoch 20/70
2822/2822 - 23s - loss: 0.0675
Epoch 21/70
2822/2822 - 23s - loss: 0.0675
Epoch 22/70
2822/2822 - 23s - loss: 0.0675
Epoch 23/70
2822/2822 - 23s - 

In [124]:
print("Predicting with neural network model ...")
y_pred_ann = nn.predict(x_test)

Predicting with neural network model ...


In [125]:
print("Preparing results for write")
nn_pred=y_pred_ann.flatten()

Preparing results for write


In [126]:
print("Neural Network predictions")
print(pd.DataFrame(nn_pred))

Neural Network predictions
                0
0       -0.011095
1       -0.014492
2        0.815096
3        0.135704
4        0.142505
...           ...
2985212  0.742349
2985213  0.742349
2985214  0.742349
2985215  0.742349
2985216  0.742349

[2985217 rows x 1 columns]


In [127]:
del train
del prop
del sample
del x_train
del x_test
del df_train
del df_test
del y_pred_ann
gc.collect()

2282

# OLS

In [138]:
np.random.seed(17)
random.seed(17)

print( "\n\nProcessing data for OLS ...")

train = pd.read_csv("/Users/kimjw/Desktop/Kaggle/week6/train_2016_v2.csv", parse_dates=["transactiondate"])
properties = pd.read_csv("/Users/kimjw/Desktop/Kaggle/week6/properties_2016.csv")
submission = pd.read_csv("/Users/kimjw/Desktop/Kaggle/week6/sample_submission.csv")
print(len(train),len(properties),len(submission))

def get_features(df):
    df["transactiondate"] = pd.to_datetime(df["transactiondate"])
    df["transactiondate_year"] = df["transactiondate"].dt.year
    df["transactiondate_month"] = df["transactiondate"].dt.month
    df['transactiondate'] = df['transactiondate'].dt.quarter
    df = df.fillna(-1.0)
    return df

def MAE(y, ypred):
    #logerror=log(Zestimate)−log(SalePrice)
    return np.sum([abs(y[i]-ypred[i]) for i in range(len(y))]) / len(y)

train = pd.merge(train, properties, how='left', on='parcelid')
y = train['logerror'].values
test = pd.merge(submission, properties, how='left', left_on='ParcelId', right_on='parcelid')
properties = [] #memory

exc = [train.columns[c] for c in range(len(train.columns)) if train.dtypes[c] == 'O'] + ['logerror','parcelid']
col = [c for c in train.columns if c not in exc]

train = get_features(train[col])
test['transactiondate'] = '2016-01-01' #should use the most common training date
test = get_features(test[col])


print("\nFitting OLS...")
reg = LinearRegression(n_jobs=-1)
reg.fit(train, y); print('fit...')
print(MAE(y, reg.predict(train)))
train = [];  y = [] #memory

test_dates = ['2016-10-01','2016-11-01','2016-12-01','2017-10-01','2017-11-01','2017-12-01']
test_columns = ['201610','201611','201612','201710','201711','201712']



Processing data for OLS ...
90275 2985217 2985217

Fitting OLS...
fit...
0.06837008219611951


# Combine Result

In [139]:
print( "\nCombining XGBoost, LightGBM, NN, and baseline predicitons ..." )
lgb_weight = 1 - XGB_WEIGHT - BASELINE_WEIGHT - NN_WEIGHT - OLS_WEIGHT 
lgb_weight0 = lgb_weight / (1 - OLS_WEIGHT)
xgb_weight0 = XGB_WEIGHT / (1 - OLS_WEIGHT)
baseline_weight0 =  BASELINE_WEIGHT / (1 - OLS_WEIGHT)
nn_weight0 = NN_WEIGHT / (1 - OLS_WEIGHT)
pred0 = 0
pred0 += xgb_weight0*xgb_pred
pred0 += baseline_weight0*BASELINE_PRED
pred0 += lgb_weight0*p_test
pred0 += nn_weight0*nn_pred

print( "\nCombined XGB/LGB/NN/baseline predictions:" )
print( pd.DataFrame(pred0).head() )

print( "\nPredicting with OLS and combining with XGB/LGB/NN/baseline predicitons: ..." )
for i in range(len(test_dates)):
    test['transactiondate'] = test_dates[i]
    pred = FUDGE_FACTOR * ( OLS_WEIGHT*reg.predict(get_features(test)) + (1-OLS_WEIGHT)*pred0 )
    submission[test_columns[i]] = [float(format(x, '.4f')) for x in pred]
    print('predict...', i)

print( "\nCombined XGB/LGB/NN/baseline/OLS predictions:" )
print( submission.head() )


Combining XGBoost, LightGBM, NN, and baseline predicitons ...

Combined XGB/LGB/NN/baseline predictions:
          0
0 -0.021495
1 -0.012497
2  0.088183
3  0.057481
4  0.020904

Predicting with OLS and combining with XGB/LGB/NN/baseline predicitons: ...
predict... 0
predict... 1
predict... 2
predict... 3
predict... 4
predict... 5

Combined XGB/LGB/NN/baseline/OLS predictions:
   ParcelId  201610  201611  201612  201710  201711  201712
0  10754147 -0.0249 -0.0249 -0.0249 -0.0249 -0.0249 -0.0249
1  10759547 -0.0158 -0.0159 -0.0159 -0.0158 -0.0159 -0.0159
2  10843547  0.1377  0.1377  0.1377  0.1377  0.1377  0.1377
3  10859147  0.0631  0.0631  0.0631  0.0631  0.0631  0.0631
4  10879947  0.0241  0.0241  0.0240  0.0241  0.0241  0.0240
